In [7]:
import os
def dataset_from_text_file(file_names):
    if isinstance(file_names, str):
        file_names = [file_names]
    for file_name in file_names:
        with open(file_name, 'r') as f:
            text = f.read()
        print(text)
    return None
data_dir = '../docs/interviews'
files = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
with open(files[0], 'r') as f:
    text = f.read()
# kapitza = dataset_from_text_file(file_names=files)

In [6]:
# !pip install langchain spacy
# !python -m spacy download en_core_web_sm
from langchain.text_splitter import SpacyTextSplitter
text_splitter = SpacyTextSplitter(chunk_size=256, chunk_overlap=256//4)

In [8]:
# functions to train and test
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
base_model, tokenizer = FastLanguageModel.from_pretrained( model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit", # "model",
    max_seq_length = 8192, dtype = None, load_in_4bit = True)
tokenizer = get_chat_template(tokenizer, chat_template = "llama-3", 
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)
FastLanguageModel.for_inference(base_model) 

def ask_llm(prompt, user, n_rep=1):
    inputs = tokenizer.apply_chat_template( [{"system": prompt,\
                                          "from": "human", \
                                          "value": user}],
    tokenize = True, add_generation_prompt = True, return_tensors = "pt").to(base_model.device)
    questions = []
    for _ in range(n_rep):
        outputs = base_model.generate(input_ids = inputs, max_new_tokens = 128, use_cache = True)
        full_output = tokenizer.batch_decode(outputs)
        answer = full_output[0].split('|end_header_id|>\n\n')[-1].rstrip('<|eot_id|>')
        questions.append(answer)
    
    return questions[0] if len(questions) == 1 else questions

/home/zipa/miniconda3/envs/unsloth_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.9.post3: Fast Llama patching. Transformers = 4.45.0.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.635 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [9]:
prompt1 = """Ниже дан текст в форме вопроса, адресованному Сергею Капице. Пожалуйста, выведи новый {text}, представляющий из себя сокращенный вопрос,перепиcанный простыми словами.
[вопрос]: {question}.
Выведи ответ в формате JSON, используя только русский язык: {"question": {text}}."""
prompt2 = """Ниже дан текст в форме вопроса, адресованному Сергею Капице и его ответ.
Пожалуйста, выведи новый {text}, представляющий из себя сокращенный ответ, сохранивший стиль. Ответ должен по смыслу чётко соответствовать вопросу.
[вопрос]: {question}
[ответ]: {answer}
Выведи ответ в формате JSON, используя только русский язык: {"answer": {answer}}."""

prompt3 = """На вход ты получил ```Текст```. Выдели несколько тем в тексте и перепиши его в виде отдельных абзацев, сохранив стиль."""
prompt4 = """На вход ты получил ```Ответ```. Придумай вопрос, подходящий к такому ответу."""
question = """Если взять поколения кому сегодня за сорок, навскидку они процитируют: "О сколько нам открытий чудных готовит просвещенья дух..." Так кто был популярнее, Вы, Сергей Петрович, или Александр Сергеевич?"""
answer = """Не преувеличивайте, Пушкин, конечно, был популярнее. Алла Пугачева была популярнее...  Да, это стихотворение стало нашим брендом. Эпиграф нашел наш режиссер Левкович. Кстати, это стихотворение никогда не было опубликовано в сочинениях Пушкина. Оно рассеяно у него в отрывках... У Натана Эйдельмана есть целое эссе на эту тему.  Название же передачи - "Очевидное-невероятное" - предложила моя помощница Железова. По-моему, до этого оно появлялось в фильмах, снятых Киевской киностудией. Сегодня нам приходится это словосочетание защищать, поскольку его уже пытались несколько раз украсть. Мы даже специально получили авторское свидетельство на название телепередачи - "Очевидное-невероятное"."""

import json

new_question = json.loads(ask_llm("", prompt1.replace("{question}", question)))['question']
new_answer = json.loads(ask_llm("", prompt2.replace("{question}", new_question).replace("{answer}", answer)))['answer']
# ask_llm("", prompt2.replace("{question}", question).replace("{answer}", answer))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [10]:
prompt3 = """На вход ты получил ```Текст```. Выдели несколько тем в тексте и перепиши его в виде отдельных абзацев, сохранив стиль."""
prompt3 = """Ниже дан текст рассуждений Сергея Капицы. Перепиши этот текст в виде набора тезисов, сохранив стиль.
[текст]: {text}.
Выведи ответ в формате JSON, используя только русский язык: {"theme0": {тезис0}, "theme1": {тезис1}, ...}. Не добавляй больше ничего к выдаче."""
ret = ask_llm("", prompt3.replace("{text}", answer))

In [15]:
# print(*[ans.strip() for ans in answer.split('.') if len(ans)], sep = '.\n')
items = text_splitter.split_text(answer)
print(len(items))

3


/home/zipa/miniconda3/envs/unsloth_env/lib/python3.11/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [22]:
question

'Если взять поколения кому сегодня за сорок, навскидку они процитируют: "О сколько нам открытий чудных готовит просвещенья дух..." Так кто был популярнее, Вы, Сергей Петрович, или Александр Сергеевич?'

In [67]:
prompt_eval = """Ниже дана пара вопрос-ответ, оцени их согласованность по логике по шкале от 0 до 5.
0 - вопрос и ответ не соотносятся друг с другом.
1 - в ответе слишком много лишних слов.
2 - в ответе много деталей, о которых никто не спрашивал.
3 - в ответе содержатся детали, не соответствующие вопросу.
4 - вопрос и ответ почти походят на диалог, но есть небольшая несогласованность, нечёткость.
5 - вопрос и ответ идеально подходят друг к другу.
[вопрос]: {question}
[ответ]: {answer}
Выведи оценку {value} в виде JSON: {"evaluation": {value}}. Не добавляй анализ и постронний текст."""
for q in [question, new_question]:
    for a in [answer, new_answer]:
        text = ask_llm("", prompt_eval.replace("{question}", q).replace("{answer}", a))
        print(text, q, a)

{"evaluation": 2} Если взять поколения кому сегодня за сорок, навскидку они процитируют: "О сколько нам открытий чудных готовит просвещенья дух..." Так кто был популярнее, Вы, Сергей Петрович, или Александр Сергеевич? Не преувеличивайте, Пушкин, конечно, был популярнее. Алла Пугачева была популярнее...  Да, это стихотворение стало нашим брендом. Эпиграф нашел наш режиссер Левкович. Кстати, это стихотворение никогда не было опубликовано в сочинениях Пушкина. Оно рассеяно у него в отрывках... У Натана Эйдельмана есть целое эссе на эту тему.  Название же передачи - "Очевидное-невероятное" - предложила моя помощница Железова. По-моему, до этого оно появлялось в фильмах, снятых Киевской киностудией. Сегодня нам приходится это словосочетание защищать, поскольку его уже пытались несколько раз украсть. Мы даже специально получили авторское свидетельство на название телепередачи - "Очевидное-невероятное".
{"evaluation": 4} Если взять поколения кому сегодня за сорок, навскидку они процитируют: "

In [49]:
new_answer

'Не преувеличивайте, Пушкин был популярнее. Алла Пугачева была популярнее, и это стихотворение стало нашим брендом.'

CHATGPT prompt

In [25]:
chat_prompt = f"""Дан вопрос:
```{question}```
Дан ответ на этот вопрос:
```{answer}```
Вопрос и ответ очень размыты.
1. Перепиши вопрос простыми словами, выдели из ответа главную мысль и перепиши с сохранением стиля.
2. Раздели ответ на набор тезисов. Для каждого тезиса подготовь простой и лаконичный вопрос. Перепиши каждый тезис в виде ответа на этот вопрос с сохранением изначального стиля. Все сущности в вопросе должны быть чётко определены, ответ может ссылаться на понятия, ясные из контекста.
3. Все пары получившихся вопросов и ответов выведи массива пар в формате json с полями "question", "answer"."""

[
    {
        "question": "Кто из вас более известен: вы или Пушкин?",
        "answer": "Не преувеличивайте, Пушкин, конечно, был популярнее. Алла Пугачева была популярнее..."
    },
    {
        "question": "Почему стихотворение стало брендом вашей передачи?",
        "answer": "Да, это стихотворение стало нашим брендом. Эпиграф нашел наш режиссер Левкович."
    },
    {
        "question": "Где можно найти полное стихотворение Пушкина?",
        "answer": "Кстати, это стихотворение никогда не было опубликовано в сочинениях Пушкина. Оно рассеяно у него в отрывках..."
    },
    {
        "question": "Есть ли исследование о стихотворении Пушкина?",
        "answer": "У Натана Эйдельмана есть целое эссе на эту тему."
    },
    {
        "question": "Кто предложил название вашей передачи 'Очевидное-невероятное'?",
        "answer": "Название же передачи - 'Очевидное-невероятное' - предложила моя помощница Железова."
    },
    {
        "question": "Где ранее использовалось это название?",
        "answer": "По-моему, до этого оно появлялось в фильмах, снятых Киевской киностудией."
    },
    {
        "question": "Почему вы защищаете название вашей передачи?",
        "answer": "Сегодня нам приходится это словосочетание защищать, поскольку его уже пытались несколько раз украсть."
    },
    {
        "question": "Как вы защищаете название вашей передачи?",
        "answer": "Мы даже специально получили авторское свидетельство на название телепередачи - 'Очевидное-невероятное'."
    }
]

In [42]:
from datasets import Dataset

def interview_dataset(file="../finetuning/int1.txt"):
    with open(file, 'r') as f:
        text = f.read()
    dataset = []
    for conv in data.split('\n\n'):
        if len(conv):
            roles = ['user', 'assistant']
            play = []
            for i_role, item in enumerate(conv.split('\n')):
                play.append({'content': item.strip('–\t').strip(' '), 'role': roles[i_role % 2]})
            dataset.append(play)    
    return Dataset.from_dict({'conversations': dataset})
    # [{'content': question, 'role': 'user'}, {'content': answer, 'role': 'assistant'}]

data = interview_dataset()

In [43]:
data

Dataset({
    features: ['conversations'],
    num_rows: 34
})